In [1]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import time
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from tests import test_prediction, test_generation
import math

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
# load all that we need

dataset = np.load('../dataset/wiki.train.npy')
fixtures_pred = np.load('../fixtures/prediction.npz')  # dev
fixtures_gen = np.load('../fixtures/generation.npy')  # dev
fixtures_pred_test = np.load('../fixtures/prediction_test.npz')  # test
fixtures_gen_test = np.load('../fixtures/generation_test.npy')  # test
vocab = np.load('../dataset/vocab.npy')

In [3]:
# data loader

class LanguageModelDataLoader(DataLoader):
    """
        TODO: Define data loader logic here
    """
    def __init__(self, dataset, batch_size, shuffle=True):
        
        self.dataset = dataset
        self.barch_size = batch_size
        self.shuffle = shuffle

    def __iter__(self):
        # concatenate your articles and build into batches
        
        size = self.dataset.shape[0]
        
        dataset_index = np.arange(size)
        
        np.random.shuffle(dataset_index)
        
        dastaset_concat = np.array([], dtype=int)
        
        dataset_local = self.dataset[dataset_index]
        
        for sentence in dataset_local:
            dastaset_concat = np.append(dastaset_concat, sentence)
            
        total_len = dastaset_concat.shape[0]
        fixed_data_count = math.ceil(total_len/70.0)
        
        one_batch = np.arange(70).reshape(1,70)
        one_batch_output = np.arange(70).reshape(1,70)
        All_batches_inputs = []
        All_batches_outputs = []

        for i in range(29653):        
            if i%64 == 0 and i > 0:      
                onebatch = one_batch[1:,:]
                onebatch_op = one_batch_output[1:,:]
                All_batches_inputs.append(onebatch)
                All_batches_outputs.append(onebatch_op)
                one_batch = np.arange(70).reshape(1,70)
                one_batch_output = np.arange(70).reshape(1,70)
    
            start = i * 70
            end = start + 70
    
            if i == 29653-1 :  
                onebatch = one_batch[1:,:]
                onebatch_op = one_batch_output[1:,:]
                All_batches_inputs.append(onebatch)
                All_batches_outputs.append(onebatch_op)
                one_batch = np.arange(70).reshape(1,70)
                one_batch_output = np.arange(70).reshape(1,70)
            else:
                one_data = np.array([dastaset_concat[start:end]])
                one_data_output = np.array([dastaset_concat[start+1:end+1]])
                one_batch = np.append(one_batch, one_data, axis=0)
                one_batch_output = np.append(one_batch_output, one_data_output, axis=0)
        
        All_batches_inputs = np.array(All_batches_inputs)
        All_batches_outputs = np.array(All_batches_outputs)
 
        return iter(zip(All_batches_inputs, All_batches_outputs))

        
        

In [4]:
# model

class LanguageModel(nn.Module):
    """
        TODO: Define your model here
    """
    def __init__(self,vocab_size,embed_size,hidden_size, nlayers):
        super(LanguageModel, self).__init__()
        
        self.vocab_size=vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.nlayers=nlayers
        self.embedding = nn.Embedding(vocab_size,embed_size) # Embedding layer
        self.rnn = nn.LSTM(input_size = embed_size,hidden_size=hidden_size,num_layers=nlayers)
        self.scoring = nn.Linear(hidden_size,vocab_size) # Projection layer

    def forward(self, seq_batch):
        # Feel free to add extra arguments to forward (like an argument to pass in the hiddens)
        
        batch_size = seq_batch.size(1)
        embed = self.embedding(seq_batch) #L x N x E
        hidden = None
        output_lstm,hidden = self.rnn(embed,hidden) #L x N x H
        output_lstm_flatten = output_lstm.view(-1,self.hidden_size) #(L*N) x H
        output_flatten = self.scoring(output_lstm_flatten) #(L*N) x V
        
        return output_flatten.view(-1,batch_size,self.vocab_size)


    


In [5]:
# model trainer

class LanguageModelTrainer:
    def __init__(self, model, loader, max_epochs=1, run_id='exp'):
        """
            Use this class to train your model
        """
        # feel free to add any other parameters here
        self.model = model
        self.loader = loader
        self.train_losses = []
        self.val_losses = []
        self.predictions = []
        self.predictions_test = []
        self.generated_logits = []
        self.generated = []
        self.generated_logits_test = []
        self.generated_test = []
        self.epochs = 0
        self.max_epochs = max_epochs
        self.run_id = run_id
        
        # TODO: Define your optimizer and criterion here
        self.optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
        self.criterion = nn.CrossEntropyLoss()

    def train(self):
        self.model.train() # set to training mode
        epoch_loss = 0
        num_batches = 0
        for batch_num, (inputs, targets) in enumerate(self.loader):
            epoch_loss += self.train_batch(inputs, targets)
        epoch_loss = epoch_loss / (batch_num + 1)
        self.epochs += 1
        print('[TRAIN]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, epoch_loss))
        self.train_losses.append(epoch_loss)

    def train_batch(self, inputs, targets):
        """ 
            TODO: Define code for training a single batch of inputs
        
        """
        self.optimizer.zero_grad()
        
        inputs = torch.LongTensor(inputs).to('cuda').transpose(0,1)
        targets = torch.LongTensor(targets).to('cuda').transpose(0,1)
        
        
        output = self.model(inputs)
        
        loss = self.criterion(output.view(-1,output.size(2)),targets.reshape(-1))
        #contiguous().view(-1)
        loss.backward()
        self.optimizer.step()
        
        batchLoss = loss.item()
        
        
        
        return batchLoss

    
    def test(self):
        # don't change these
        self.model.eval() # set to eval mode
        predictions = TestLanguageModel.prediction(fixtures_pred['inp'], self.model) # get predictions
        self.predictions.append(predictions)
        generated_logits = TestLanguageModel.generation(fixtures_gen, 10, self.model) # generated predictions for 10 words
        generated_logits_test = TestLanguageModel.generation(fixtures_gen_test, 10, self.model)
        nll = test_prediction(predictions, fixtures_pred['out'])
        generated = test_generation(fixtures_gen, generated_logits, vocab)
        generated_test = test_generation(fixtures_gen_test, generated_logits_test, vocab)
        self.val_losses.append(nll)
        
        self.generated.append(generated)
        self.generated_test.append(generated_test)
        self.generated_logits.append(generated_logits)
        self.generated_logits_test.append(generated_logits_test)
        
        # generate predictions for test data
        predictions_test = TestLanguageModel.prediction(fixtures_pred_test['inp'], self.model) # get predictions
        self.predictions_test.append(predictions_test)
            
        print('[VAL]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, nll))
        return nll

    def save(self):
        # don't change these
        model_path = os.path.join('experiments', self.run_id, 'model-{}.pkl'.format(self.epochs))
        torch.save({'state_dict': self.model.state_dict()},
            model_path)
        np.save(os.path.join('experiments', self.run_id, 'predictions-{}.npy'.format(self.epochs)), self.predictions[-1])
        np.save(os.path.join('experiments', self.run_id, 'predictions-test-{}.npy'.format(self.epochs)), self.predictions_test[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-{}.npy'.format(self.epochs)), self.generated_logits[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-test-{}.npy'.format(self.epochs)), self.generated_logits_test[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}-test.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated_test[-1])


In [6]:
class TestLanguageModel:
    def prediction(inp, model):
        """
            TODO: write prediction code here
            
            :param inp:
            :return: a np.ndarray of logits
        """
        input = torch.LongTensor(inp).to('cuda').transpose(0,1)
        
        out = model(input)
        
        out = out[-1].to('cpu').detach().numpy()
        
        return out

        
    def generation(inp, forward, model):
        """
            TODO: write generation code here

            Generate a sequence of words given a starting sequence.
            :param inp: Initial sequence of words (batch size, length)
            :param forward: number of additional words to generate
            :return: generated words (batch size, forward)
        """
        new_words = []
        
        input = torch.LongTensor(inp).to('cuda').transpose(0,1)
        out = model(input)
        
        out = out[-1]
        
        word_nxt = out.argmax(dim = -1)
        
        new_words.append(word_nxt.to('cpu').detach().numpy().T)
        
        for i in range(1,forward):
            input_word = word_nxt.unsqueeze(0)
            out = model(input_word)
            out = out[-1]
            word_nxt = out.argmax(dim = -1)
            wn = word_nxt.to('cpu').detach().numpy().T
#             print(wn)
            new_words.append(wn)
            
        #print(new_words.shape)
        return np.transpose(np.array(new_words))

        

In [7]:
# TODO: define other hyperparameters here

NUM_EPOCHS = 10
BATCH_SIZE = 100


In [8]:
run_id = str(int(time.time()))
if not os.path.exists('./experiments'):
    os.mkdir('./experiments')
os.mkdir('./experiments/%s' % run_id)
print("Saving models, predictions, and generated words to ./experiments/%s" % run_id)

Saving models, predictions, and generated words to ./experiments/1575870224


In [9]:
model = LanguageModel(vocab_size=len(vocab), embed_size= 400, hidden_size=1150,nlayers=3 )
model = model.to(device)
loader = LanguageModelDataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True, )
trainer = LanguageModelTrainer(model=model, loader=loader, max_epochs=NUM_EPOCHS, run_id=run_id)

In [ ]:
# nll = trainer.test()

In [10]:
best_nll = 1e30 
for epoch in range(NUM_EPOCHS):
    print("EPOCH: ", epoch)
    
    trainer.train()
    nll = trainer.test()
    if nll < best_nll:
        best_nll = nll
        print("Saving model, predictions and generated output for epoch "+str(epoch)+" with NLL: "+ str(best_nll))
        trainer.save()
    

EPOCH:  0
[TRAIN]  Epoch [2/10]   Loss: 7.5967
[VAL]  Epoch [2/10]   Loss: 6.7736
Saving model, predictions and generated output for epoch 0 with NLL: 6.7735925
EPOCH:  1
[TRAIN]  Epoch [3/10]   Loss: 7.1118
[VAL]  Epoch [3/10]   Loss: 5.9422
Saving model, predictions and generated output for epoch 1 with NLL: 5.942178
EPOCH:  2
[TRAIN]  Epoch [4/10]   Loss: 6.1310
[VAL]  Epoch [4/10]   Loss: 5.2116
Saving model, predictions and generated output for epoch 2 with NLL: 5.211562
EPOCH:  3
[TRAIN]  Epoch [5/10]   Loss: 5.6424
[VAL]  Epoch [5/10]   Loss: 4.9985
Saving model, predictions and generated output for epoch 3 with NLL: 4.9984617
EPOCH:  4
[TRAIN]  Epoch [6/10]   Loss: 5.3280
[VAL]  Epoch [6/10]   Loss: 4.7880
Saving model, predictions and generated output for epoch 4 with NLL: 4.787966
EPOCH:  5
[TRAIN]  Epoch [7/10]   Loss: 5.0547
[VAL]  Epoch [7/10]   Loss: 4.6945
Saving model, predictions and generated output for epoch 5 with NLL: 4.694546
EPOCH:  6
[TRAIN]  Epoch [8/10]   Loss

KeyboardInterrupt: 

In [ ]:
# Don't change these
# plot training curves
plt.figure()
plt.plot(range(1, trainer.epochs + 1), trainer.train_losses, label='Training losses')
plt.plot(range(1, trainer.epochs + 1), trainer.val_losses, label='Validation losses')
plt.xlabel('Epochs')
plt.ylabel('NLL')
plt.legend()
plt.show()

In [ ]:
# see generated output
print (trainer.generated[-1]) # get last generated output